In [5]:
import openai
import os

from dotenv import load_dotenv
load_dotenv()

from langchain.docstore.document import Document

from youtube_transcript_api import YouTubeTranscriptApi

In [6]:
def read_youtube(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)

        # Convert to text
        transcript = ' '.join([t['text'] for t in transcript])

        # Create document
        document = Document(page_content=transcript, metadata={'source': f"https://www.youtube.com/watch?v={video_id}"})

        return document
    except:
        return None

In [7]:
video_id = 'O0dUOtOIrfs'

In [12]:
text = read_youtube(video_id)

In [9]:
model="gpt-3.5-turbo-1106"

In [10]:
from llama_index.llms import OpenAI
llm = OpenAI(model=model, temperature=0)



In [11]:
from llama_index import Prompt

text_qa_template = Prompt(
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

refine_template = Prompt(
    "We have the opportunity to refine the original answer "
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "Given the new context, refine the original answer to better "
    "answer the question: {query_str}. "
    "If the context isn't useful, output the original answer again.\n"
    "Original Answer: {existing_answer}"
)

In [14]:
question = "What is a pipe operator?"
prompt = text_qa_template.format(context_str=text, query_str=question)
response = llm.complete(prompt)
print(response.text)


A pipe operator is a symbol used in the Lang chain expression language to string together different components or functions in a chain. It allows the output from one component to be passed as input to the next component, simplifying the process of building chains within Lang chain. The pipe operator is represented by the "|" symbol and is used to create a minimalist and flexible approach to chaining components together.


In [16]:
question = "How do I create an Lanchain? Write a sample code."
prompt = text_qa_template.format(context_str=text, query_str=question)
response_gen = llm.stream_complete(prompt)
for response in response_gen:
    print(response.delta, end="")

To create a Langchain, you can use the expression language and the pipe operator to string together different components. Here's a sample code to create a Langchain using the expression language:

```python
from langchain import Runnable

# Define the functions to be used in the Langchain
def add_five(num):
    return num + 5

def multiply_by_two(num):
    return num * 2

# Create runnable lambdas for the functions
add_five_runnable = Runnable(lambda x: add_five(x))
multiply_by_two_runnable = Runnable(lambda x: multiply_by_two(x))

# String the components together using the pipe operator
result = add_five_runnable | multiply_by_two_runnable

# Invoke the Langchain
output = result.invoke(3)

# Print the output
print(output)  # Output will be 16
```

In this sample code, we first define the functions `add_five` and `multiply_by_two`. Then, we create runnable lambdas for these functions and string them together using the pipe operator. Finally, we invoke the Langchain with an input value 

In [17]:
system_message = """
    You are a note taking assistant for a courses. 
    
    Given the Document, write a note based on the following format and instructions:
    
    ## ABSTRACT
    - one paragraph summarizing the main concepts 
    - emphasize critical points or key takeaways.
    - Use bold or italic text to highlight these.

    ## KEY POINTS
    - Include important terms and their meanings.
    - Break the topic into smaller sections.
    - Each section should focus on a specific aspect of the topic.
    - Use bullet points or numbered lists for clarity.
    
    ## CONTEXT
    - The context should focus on the details of the document, should be well structured, informative, in depth, with facts and numbers if available and a minimum of 300 words.
    - Provide examples to illustrate how concepts are applied.
    - You should strive to write the context as long as you can using all relevant and necessary information provided.
    - You must write the context in bullet form.
    - You MUST determine your own concrete and valid opinion based on the given information. Do NOT deter to general and meaningless conclusions.
    
    ## REFLECTIONS
    - Formulate questions that test understanding of the topic.
    - Include space for reflections or personal notes.
    - Recap the most important points
    
    """


In [20]:
text.page_content

"today we're going to be talking about Lang chain expression language which is a pretty interesting idea that essentially allows us to write very minimalist code to build chains within line chain and for sure I think we'll see from this video we can use a lot of L chains more advanced features like parallel execution async and streaming very easily using the expression language rather than just the more typical approach to build Lang chain chains and in my opinion it's worth trying just for that I think we'll see that just using this you can build stuff very quickly that's not to say it doesn't have its cons but we'll dive into those later so let's just begin with what this expression language actually is so there's a page here in the line train dos talking about this expression language right so it's LC for short and yeah they just explain a few things you know we streaming acing parel execution so on and so on right but let's just jump into this notebook and we'll see more of how thi

In [21]:
from llama_index.llms import ChatMessage

chat_history = [
    ChatMessage(role="system", content=system_message),
    ChatMessage(role="user", content=text.page_content),
]

response = llm.chat(chat_history)
print(response.message)



assistant: ## ABSTRACT
The Lang chain expression language allows for minimalist code to build chains within line chain, offering advanced features like parallel execution, async, and streaming. The expression language provides super fast development of chains, easy integration with other Lang chain products, and out-of-the-box support for advanced features. The pipe operator is a key feature of the expression language, allowing for a more flexible and simpler way to string components together.

## KEY POINTS
- Lang chain expression language: Allows for minimalist code to build chains within line chain, offering advanced features like parallel execution, async, and streaming.
- Pipe operator: Enables the stringing together of components in a more flexible and simpler way.
- Runnable parallel: Allows for running multiple components in parallel and extracting multiple values from them.
- Runnable lambdas: Provides an abstraction for creating and running custom functions within the express